In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 1: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = './log/star/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 250
test_batch = 250
lr = 0.1
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/star/128/b0/to_style1/2000shot/self2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '__'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'style1/2000_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/star/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 200, gamma=0.1, last_epoch=-1)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=2, total_epoch=50, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp

        # measure accuracy and record loss
        losses.update(loss.data.tolist(), inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

In [20]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
#     adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_auroc, test_auroc, source_auroc])
    is_best = test_auroc+source_auroc > best_acc
    best_acc = max(test_auroc+source_auroc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    scheduler.step()
    
    if (epoch+1)%500 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.100000
Train | 16/16 | Loss:1.4161 | MainLoss:0.9489 | Alpha:0.0513 | SPLoss:4.6723 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.6939 | MainLoss:0.6939 | SPLoss:4.5847 | CLSLoss:0.0000 | AUROC:0.5255
Test | 122/16 | Loss:0.6952 | MainLoss:0.6952 | SPLoss:4.5847 | CLSLoss:0.0000 | AUROC:0.4309

Epoch: [2 | 1000] LR: 0.102000
Train | 16/16 | Loss:1.0900 | MainLoss:0.6966 | Alpha:0.0519 | SPLoss:3.9336 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.6928 | MainLoss:0.6928 | SPLoss:3.2684 | CLSLoss:0.0000 | AUROC:0.5479
Test | 122/16 | Loss:0.6935 | MainLoss:0.6935 | SPLoss:3.2684 | CLSLoss:0.0000 | AUROC:0.5067

Epoch: [3 | 1000] LR: 0.104000
Train | 16/16 | Loss:0.9729 | MainLoss:0.6935 | Alpha:0.0513 | SPLoss:2.7939 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.6972 | MainLoss:0.6972 | SPLoss:2.3170 | CLSLoss:0.0000 | AUROC:0.5738
Test | 122/16 | Loss:0.6950 | MainLoss:0.6950 | SPLoss:2.3170 | CLSLoss:0.0000 | AUROC:0.6483

Epoch: [4 | 100

Train | 16/16 | Loss:0.6015 | MainLoss:0.3953 | Alpha:0.0521 | SPLoss:2.0625 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.2536 | MainLoss:0.2536 | SPLoss:2.0721 | CLSLoss:0.0000 | AUROC:0.9626
Test | 122/16 | Loss:0.4642 | MainLoss:0.4642 | SPLoss:2.0721 | CLSLoss:0.0000 | AUROC:0.8787

Epoch: [27 | 1000] LR: 0.152000
Train | 16/16 | Loss:0.5467 | MainLoss:0.3401 | Alpha:0.0536 | SPLoss:2.0656 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.3264 | MainLoss:0.3264 | SPLoss:2.0578 | CLSLoss:0.0000 | AUROC:0.9626
Test | 122/16 | Loss:0.5064 | MainLoss:0.5064 | SPLoss:2.0578 | CLSLoss:0.0000 | AUROC:0.8522

Epoch: [28 | 1000] LR: 0.154000
Train | 16/16 | Loss:0.5819 | MainLoss:0.3682 | Alpha:0.0507 | SPLoss:2.1371 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.3572 | MainLoss:0.3572 | SPLoss:2.0923 | CLSLoss:0.0000 | AUROC:0.9541
Test | 122/16 | Loss:0.4595 | MainLoss:0.4595 | SPLoss:2.0923 | CLSLoss:0.0000 | AUROC:0.8792

Epoch: [29 | 1000] LR: 0.156000
Train | 16/16

Train | 16/16 | Loss:7.0922 | MainLoss:0.3453 | Alpha:0.0513 | SPLoss:67.4697 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.4692 | MainLoss:0.4692 | SPLoss:46.7549 | CLSLoss:0.0000 | AUROC:0.9518
Test | 122/16 | Loss:0.5771 | MainLoss:0.5771 | SPLoss:46.7550 | CLSLoss:0.0000 | AUROC:0.9276

Epoch: [52 | 1000] LR: 0.200000
Train | 16/16 | Loss:3.9595 | MainLoss:0.4030 | Alpha:0.0531 | SPLoss:35.5657 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.2527 | MainLoss:0.2527 | SPLoss:24.9995 | CLSLoss:0.0000 | AUROC:0.9639
Test | 122/16 | Loss:0.5696 | MainLoss:0.5696 | SPLoss:24.9995 | CLSLoss:0.0000 | AUROC:0.8122

Epoch: [53 | 1000] LR: 0.200000
Train | 16/16 | Loss:2.2322 | MainLoss:0.3251 | Alpha:0.0518 | SPLoss:19.0707 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.5536 | MainLoss:0.5536 | SPLoss:13.9439 | CLSLoss:0.0000 | AUROC:0.9564
Test | 122/16 | Loss:0.7276 | MainLoss:0.7276 | SPLoss:13.9439 | CLSLoss:0.0000 | AUROC:0.7564

Epoch: [54 | 1000] LR: 0.199998
Trai

Train | 16/16 | Loss:0.5832 | MainLoss:0.3453 | Alpha:0.0533 | SPLoss:2.3786 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.5194 | MainLoss:0.5194 | SPLoss:2.1638 | CLSLoss:0.0000 | AUROC:0.9530
Test | 122/16 | Loss:0.7576 | MainLoss:0.7576 | SPLoss:2.1638 | CLSLoss:0.0000 | AUROC:0.8939

Epoch: [77 | 1000] LR: 0.199692
Train | 16/16 | Loss:0.5857 | MainLoss:0.3527 | Alpha:0.0510 | SPLoss:2.3302 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.2489 | MainLoss:0.2489 | SPLoss:2.3765 | CLSLoss:0.0000 | AUROC:0.9647
Test | 122/16 | Loss:0.6648 | MainLoss:0.6648 | SPLoss:2.3765 | CLSLoss:0.0000 | AUROC:0.7645

Epoch: [78 | 1000] LR: 0.199667
Train | 16/16 | Loss:0.5627 | MainLoss:0.3281 | Alpha:0.0505 | SPLoss:2.3455 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.2456 | MainLoss:0.2456 | SPLoss:2.2027 | CLSLoss:0.0000 | AUROC:0.9646
Test | 122/16 | Loss:0.5988 | MainLoss:0.5988 | SPLoss:2.2027 | CLSLoss:0.0000 | AUROC:0.8518

Epoch: [79 | 1000] LR: 0.199640
Train | 16/16

Test | 122/16 | Loss:0.7485 | MainLoss:0.7485 | SPLoss:174.7641 | CLSLoss:0.0000 | AUROC:0.6561

Epoch: [101 | 1000] LR: 0.198817
Train | 16/16 | Loss:13.4233 | MainLoss:0.3634 | Alpha:0.0495 | SPLoss:130.5983 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.3174 | MainLoss:0.3174 | SPLoss:90.1826 | CLSLoss:0.0000 | AUROC:0.9658
Test | 122/16 | Loss:0.7495 | MainLoss:0.7495 | SPLoss:90.1826 | CLSLoss:0.0000 | AUROC:0.8661

Epoch: [102 | 1000] LR: 0.198769
Train | 16/16 | Loss:7.1439 | MainLoss:0.3602 | Alpha:0.0499 | SPLoss:67.8368 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.2909 | MainLoss:0.2909 | SPLoss:47.4155 | CLSLoss:0.0000 | AUROC:0.9603
Test | 122/16 | Loss:0.6814 | MainLoss:0.6814 | SPLoss:47.4155 | CLSLoss:0.0000 | AUROC:0.7437

Epoch: [103 | 1000] LR: 0.198719
Train | 16/16 | Loss:3.8901 | MainLoss:0.3137 | Alpha:0.0515 | SPLoss:35.7640 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.2709 | MainLoss:0.2709 | SPLoss:25.0972 | CLSLoss:0.0000 | AUROC:0.966

...

Test | 122/16 | Loss:0.5199 | MainLoss:0.5199 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9021

Epoch: [901 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.3632 | MainLoss:0.1875 | Alpha:0.0507 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1645 | MainLoss:0.1645 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9844
Test | 122/16 | Loss:0.5199 | MainLoss:0.5199 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9020

Epoch: [902 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.3518 | MainLoss:0.1761 | Alpha:0.0504 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1645 | MainLoss:0.1645 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9842
Test | 122/16 | Loss:0.5199 | MainLoss:0.5199 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9021

Epoch: [903 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.3547 | MainLoss:0.1790 | Alpha:0.0520 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1645 | MainLoss:0.1645 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9841
Test | 122

Test | 122/16 | Loss:0.5199 | MainLoss:0.5199 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9021

Epoch: [926 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.3455 | MainLoss:0.1698 | Alpha:0.0493 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1645 | MainLoss:0.1645 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9844
Test | 122/16 | Loss:0.5199 | MainLoss:0.5199 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9022

Epoch: [927 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.3392 | MainLoss:0.1635 | Alpha:0.0516 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1645 | MainLoss:0.1645 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9844
Test | 122/16 | Loss:0.5199 | MainLoss:0.5199 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9024

Epoch: [928 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.3376 | MainLoss:0.1619 | Alpha:0.0519 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1645 | MainLoss:0.1645 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9843
Test | 122

Test | 122/16 | Loss:0.5200 | MainLoss:0.5200 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9023

Epoch: [951 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.3468 | MainLoss:0.1711 | Alpha:0.0506 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1644 | MainLoss:0.1644 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9847
Test | 122/16 | Loss:0.5200 | MainLoss:0.5200 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9022

Epoch: [952 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.3464 | MainLoss:0.1707 | Alpha:0.0521 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1644 | MainLoss:0.1644 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9842
Test | 122/16 | Loss:0.5200 | MainLoss:0.5200 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9022

Epoch: [953 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.3385 | MainLoss:0.1628 | Alpha:0.0492 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1644 | MainLoss:0.1644 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9842
Test | 122

Test | 122/16 | Loss:0.5200 | MainLoss:0.5200 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9022

Epoch: [976 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.3372 | MainLoss:0.1615 | Alpha:0.0493 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1645 | MainLoss:0.1645 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9841
Test | 122/16 | Loss:0.5200 | MainLoss:0.5200 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9021

Epoch: [977 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.3288 | MainLoss:0.1531 | Alpha:0.0498 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1645 | MainLoss:0.1645 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9839
Test | 122/16 | Loss:0.5200 | MainLoss:0.5200 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9023

Epoch: [978 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.3629 | MainLoss:0.1872 | Alpha:0.0488 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.0000
Test | 31/16 | Loss:0.1645 | MainLoss:0.1645 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9839
Test | 122

Test | 122/16 | Loss:0.5200 | MainLoss:0.5200 | SPLoss:1.7570 | CLSLoss:0.0000 | AUROC:0.9020
